Добро пожаловать!
Мой  проект по изучению сильных и модальных глаголов в немецком языке представляет собой важный шаг в освоении этого сложного, но прекрасного языка.
Цель проекта: Главная цель этого проекта - помочь вам лучше понять и использовать сильные и модальные глаголы в немецком языке. Это ключевой элемент грамматики, который поможет вам строить более сложные и точные предложения.

In [ ]:
pip install telebot

In [ ]:
pip install gTTS

In [3]:
 #рабочий бот объединенный учить глаголы и учить глаголы в предложениях
import telebot
from telebot import types
import pandas as pd
from gtts import gTTS
import os
import random
import pyttsx3
import datetime

bot = telebot.TeleBot("TOKEN")


# Загрузите ваши данные здесь
data3 = pd.read_excel('data3.xlsx', header=0)
data3 = data3.rename(columns={5: 'level'})
user_data = {}  # Определение user_data здесь

@bot.message_handler(commands=['start'])
def send_welcome(message):
    markup = types.ReplyKeyboardMarkup(row_width=2)
    itembtn1 = types.KeyboardButton('Учить глаголы')
    itembtn2 = types.KeyboardButton('Учить глаголы в предложениях')
    markup.add(itembtn1, itembtn2)
    bot.send_message(message.chat.id, "Привет! Давай начнем изучение сильных глаголов. Выберите, что вы хотите учить:", reply_markup=markup)

@bot.message_handler(func=lambda message: message.text == 'Учить глаголы')
def learn_verbs(message):
    markup = types.ReplyKeyboardMarkup(row_width=2)
    itembtn1 = types.KeyboardButton('A1')
    itembtn2 = types.KeyboardButton('A2')
    itembtn3 = types.KeyboardButton('B1')
    itembtn4 = types.KeyboardButton('B2')
    itembtn5 = types.KeyboardButton('C1')
    itembtn6 = types.KeyboardButton('Модальные глаголы')  # Новая кнопка
    markup.add(itembtn1, itembtn2, itembtn3, itembtn4, itembtn5, itembtn6)
    bot.send_message(message.chat.id, "Выберите свой уровень немецкого языка или изучение модальных глаголов", reply_markup=markup)
    bot.register_next_step_handler(message, set_level)

def set_level(message):
    global df  # Убедитесь, что df глобально доступен
    df = pd.read_excel('data3.xlsx', header=0)
    df = df.rename(columns={5: 'level'})

    user_data[message.chat.id] = {'level': message.text}
    next_question(message)  # Добавьте эту строку
    
def next_question(message):
    chosen_level = user_data[message.chat.id]['level']
    data_for_level = df[df['level'] == chosen_level]

    if not data_for_level.empty:
        row = data_for_level.sample(1)
        user_data[message.chat.id]['current_row'] = row
        question = row.iloc[0, 4]
        correct_answer = ', '.join(row.iloc[0, [0, 2, 3]].values)
        wrong_answers = data_for_level.drop(row.index).sample(2)
        wrong_answers = [', '.join(wrong_answers.iloc[i, [0, 2, 3]].values) for i in range(2)]
        user_data[message.chat.id]['correct_answer'] = correct_answer

        # Перемешиваем варианты ответов
        answers = [correct_answer] + wrong_answers
        random.shuffle(answers)

        markup = types.ReplyKeyboardMarkup(row_width=2)
        itembtns = [types.KeyboardButton(answer) for answer in answers]
        itembtns.append(types.KeyboardButton('Закончить'))
        markup.add(*itembtns)

        bot.send_message(message.chat.id, f"Глагол: {question}", reply_markup=markup)
        bot.register_next_step_handler(message, check_answer)
    else:
        bot.send_message(message.chat.id, "Извините, но у меня нет вопросов для этого уровня. Нажмите /start")


def text_to_speech(text, lang):
    tts = gTTS(text=text, lang=lang)
    filename = 'voice.mp3'
    tts.save(filename)
    return filename
        
        
def check_answer(message):
    if message.text.lower() == 'закончить':
        markup = types.InlineKeyboardMarkup()
        #markup.add(types.InlineKeyboardButton(text='Старт', callback_data='ok'))
        bot.send_message(message.chat.id, "Обучение завершено. Спасибо за участие! Хотите начать заново, нажмите кнопку /start", reply_markup=markup)
    elif message.text.lower() == user_data[message.chat.id]['correct_answer'].lower():
        bot.reply_to(message, "Правильно! Это было довольно легко. Давай продолжим.")
        filename = text_to_speech(user_data[message.chat.id]['correct_answer'], 'de')  # Преобразуем текст в аудио
        with open(filename, 'rb') as audio:  # Открываем аудиофайл
            bot.send_audio(message.chat.id, audio=audio)  # Отправляем аудиофайл
        os.remove(filename)  # Удаляем аудиофайл
        next_question(message)  # Перезапускаем обучение с выбором уровня
    else:
        bot.reply_to(message, "Увы, это неправильный ответ. Попробуй еще раз. ")
        bot.register_next_step_handler(message, check_answer)  # Здесь мы снова регистрируем check_answer, а не next_question




@bot.message_handler(func=lambda message: message.text == 'Учить глаголы в предложениях')
def learn_verbs_in_sentences(message):
    markup = types.ReplyKeyboardMarkup(row_width=2)
    itembtn1 = types.KeyboardButton('A1')
    itembtn2 = types.KeyboardButton('A2')
    itembtn3 = types.KeyboardButton('B1')
    itembtn4 = types.KeyboardButton('B2')
    itembtn5 = types.KeyboardButton('C1')
    markup.add(itembtn1, itembtn2, itembtn3, itembtn4, itembtn5)
    bot.send_message(message.chat.id, "Привет! Давай начнем изучение сильных глаголов в предложениях. Выберите свой уровень немецкого языка", reply_markup=markup)
    bot.register_next_step_handler(message, echo_all)



@bot.message_handler(func=lambda message: True)
def echo_all(message):
    if message.text in ['A1', 'A2', 'B1', 'B2', 'C1']:
        user_data[message.chat.id] = {'level': message.text}
        markup = types.ReplyKeyboardMarkup(row_width=2)
        itembtn1 = types.KeyboardButton('Простые предложения')
        itembtn2 = types.KeyboardButton('Сложные предложения')
        itembtn3 = types.KeyboardButton('Закончить обучение')  # Добавляем кнопку "Закончить обучение"
        markup.add(itembtn1, itembtn2, itembtn3)
        bot.send_message(message.chat.id, f"Выбери сложность предложений", reply_markup=markup)

    elif message.text.lower() == 'закончить обучение':
        bot.send_message(message.chat.id, "Обучение завершено. Спасибо за участие! Хотите начать заново, нажмите кнопку /start")

    elif message.text == 'Простые предложения':
        level = user_data[message.chat.id]['level']
        data = data3[(data3['level'] == level)]
        for _ in range(1):  # Повторяем 1 раз
            row = data.sample(1)
            german_sentence = f"{row.iloc[0, 0]} {row.iloc[0, 1]} {row.iloc[0, 2]} {row.iloc[0, 3]}"
            translation = row.iloc[0, 4]

            # Отправляем немецкое предложение
            bot.send_message(message.chat.id, f"Глагол: *{row.iloc[0, 0]}* // *{row.iloc[0, 1]}* // {row.iloc[0, 2]} // *{row.iloc[0, 3]}*. Перевод: {translation}", parse_mode='Markdown')

            # Отправляем русское предложение
            for i in range(7, 14, 6):
                for j in range(2):
                    answer = row.iloc[0, i+j]
                    # Находим глаголы в предложении
                    verb1 = row.iloc[0, 1]
                    verb2 = row.iloc[0, 3]
                    # Заменяем глаголы на глаголы, выделенные жирным шрифтом
                    answer = answer.replace(verb1, f"*{verb1}*", 1)
                    answer = answer.replace(verb2, f"*{verb2}*", 1)
                    bot.send_message(message.chat.id, answer, parse_mode='Markdown')




                  # Создаем аудиофайл после отправки немецкого предложения
            #german_sentence = row.iloc[0, 13]  # Получаем немецкое предложение из 14-го столбца
            tts = gTTS(text=row.iloc[0, 13], lang='de')
            filename = "Partizip II (прошлое).mp3"
            tts.save(filename)
            audio = open(filename, 'rb')
            bot.send_audio(message.chat.id, audio)
            audio.close()
            os.remove(filename)

    elif message.text == 'Сложные предложения':
        level = user_data[message.chat.id]['level']
        data = data3[(data3['level'] == level)]
        for _ in range(1):  # Повторяем 1 раз
            row = data.sample(1)
            german_sentence = f"{row.iloc[0, 0]} {row.iloc[0, 1]} {row.iloc[0, 2]} {row.iloc[0, 3]}"
            translation = row.iloc[0, 4]

            # Отправляем немецкое предложение
            # Отправляем немецкое предложение
            bot.send_message(message.chat.id, f"Глагол: *{row.iloc[0, 0]}* // *{row.iloc[0, 1]}* // {row.iloc[0, 2]} // *{row.iloc[0, 3]}*. Перевод: {translation}", parse_mode='Markdown')

                    # Отправляем русское предложение
            for i in range(16, 24, 6):
                for j in range(2):
                    answer = row.iloc[0, i+j]
                    # Находим глаголы в предложении
                    verb1 = row.iloc[0, 1]
                    verb2 = row.iloc[0, 3]
                    # Заменяем глаголы на глаголы, выделенные жирным шрифтом
                    answer = answer.replace(verb1, f"*{verb1}*", 1)
                    answer = answer.replace(verb2, f"*{verb2}*", 1)
                    bot.send_message(message.chat.id, answer, parse_mode='Markdown')
                    
                    
                  # Создаем аудиофайл после отправки немецкого предложения
            #german_sentence = row.iloc[0, 13]  # Получаем немецкое предложение из 14-го столбца
            tts = gTTS(text=row.iloc[0, 22], lang='de')
            filename = "Partizip II (прошлое).mp3"
            tts.save(filename)
            audio = open(filename, 'rb')
            bot.send_audio(message.chat.id, audio)
            audio.close()
            os.remove(filename)

bot.polling(none_stop=True)
    
    
    
    
    


In [ ]:
 #работает 1я чать учить глаголы
import telebot
from telebot import types
import pandas as pd
from gtts import gTTS
import os
import random
import pyttsx3
import datetime

bot = telebot.TeleBot("TOKEN")


# Загрузите ваши данные здесь
data3 = pd.read_excel('data3.xlsx', header=0)
data3 = data3.rename(columns={5: 'level'})
user_data = {}  # Определение user_data здесь

@bot.message_handler(commands=['start'])
def send_welcome(message):
    markup = types.ReplyKeyboardMarkup(row_width=2)
    itembtn1 = types.KeyboardButton('Учить глаголы')
    itembtn2 = types.KeyboardButton('Учить глаголы в предложениях')
    markup.add(itembtn1, itembtn2)
    bot.send_message(message.chat.id, "Привет! Давай начнем изучение сильных глаголов. Выберите, что вы хотите учить:", reply_markup=markup)

@bot.message_handler(func=lambda message: message.text == 'Учить глаголы')
def learn_verbs(message):
    markup = types.ReplyKeyboardMarkup(row_width=2)
    itembtn1 = types.KeyboardButton('A1')
    itembtn2 = types.KeyboardButton('A2')
    itembtn3 = types.KeyboardButton('B1')
    itembtn4 = types.KeyboardButton('B2')
    itembtn5 = types.KeyboardButton('C1')
    itembtn6 = types.KeyboardButton('Модальные глаголы')  # Новая кнопка
    markup.add(itembtn1, itembtn2, itembtn3, itembtn4, itembtn5, itembtn6)
    bot.send_message(message.chat.id, "Привет! Давай начнем изучение сильных глаголов. Выберите свой уровень немецкого языка или изучение модальных глаголов", reply_markup=markup)
    bot.register_next_step_handler(message, set_level)

def set_level(message):
    global df  # Убедитесь, что df глобально доступен
    df = pd.read_excel('data3.xlsx', header=0)
    df = df.rename(columns={5: 'level'})

    user_data[message.chat.id] = {'level': message.text}
    next_question(message)  # Добавьте эту строку
    
def next_question(message):
    chosen_level = user_data[message.chat.id]['level']
    data_for_level = df[df['level'] == chosen_level]

    if not data_for_level.empty:
        row = data_for_level.sample(1)
        user_data[message.chat.id]['current_row'] = row
        question = row.iloc[0, 4]
        correct_answer = ', '.join(row.iloc[0, [0, 2, 3]].values)
        wrong_answers = data_for_level.drop(row.index).sample(2)
        wrong_answers = [', '.join(wrong_answers.iloc[i, [0, 2, 3]].values) for i in range(2)]
        user_data[message.chat.id]['correct_answer'] = correct_answer

        # Перемешиваем варианты ответов
        answers = [correct_answer] + wrong_answers
        random.shuffle(answers)

        markup = types.ReplyKeyboardMarkup(row_width=2)
        itembtns = [types.KeyboardButton(answer) for answer in answers]
        itembtns.append(types.KeyboardButton('Закончить'))
        markup.add(*itembtns)

        bot.send_message(message.chat.id, f"Глагол: {question}", reply_markup=markup)
        bot.register_next_step_handler(message, check_answer)
    else:
        bot.send_message(message.chat.id, "Извините, но у меня нет вопросов для этого уровня. Нажмите /start")
        
        
def check_answer(message):
    if message.text.lower() == 'закончить':
        markup = types.InlineKeyboardMarkup()
        #markup.add(types.InlineKeyboardButton(text='Старт', callback_data='ok'))
        bot.send_message(message.chat.id, "Обучение завершено. Спасибо за участие! Хотите начать заново, нажмите кнопку /start", reply_markup=markup)
    elif message.text.lower() == user_data[message.chat.id]['correct_answer'].lower():
        bot.reply_to(message, "Правильно! Это было довольно легко. Давай продолжим.")
        filename = text_to_speech(user_data[message.chat.id]['correct_answer'], 'de')  # Преобразуем текст в аудио
        with open(filename, 'rb') as audio:  # Открываем аудиофайл
            bot.send_audio(message.chat.id, audio=audio)  # Отправляем аудиофайл
        os.remove(filename)  # Удаляем аудиофайл
        next_question(message)  # Перезапускаем обучение с выбором уровня
    else:
        bot.reply_to(message, "Увы, это неправильный ответ. Попробуй еще раз. ")
        bot.register_next_step_handler(message, check_answer)  # Здесь мы снова регистрируем check_answer, а не next_question





bot.polling(none_stop=True)
    